<a href="https://colab.research.google.com/github/hannaribaspeeters/NLP-ETH/blob/main/neural_tranducers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 - Transliteration

### Setup [do not change]

In [ ]:
!git clone https://github.com/butoialexandra/eth-nlp-f22-hw3.git

import os
os.chdir('eth-nlp-f22-hw3')

!pip install -e .

Cloning into 'eth-nlp-f22-hw3'...
remote: Enumerating objects: 29, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 29 (delta 6), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (29/29), 9.90 KiB | 1.10 MiB/s, done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/eth-nlp-f22-hw3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 3.0 MB/s eta 0:00:00
  Running setup.py develop for rayuela


In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC' -O 'test_cases.pkl'

--2023-02-07 20:51:16--  https://docs.google.com/uc?export=download&id=1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC
Resolving docs.google.com (docs.google.com)... 108.177.126.113, 108.177.126.139, 108.177.126.102, ...
Connecting to docs.google.com (docs.google.com)|108.177.126.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0ca9nl6mqdklkahls8l7kq3ueperu7gi/1675803075000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=11aecb84-0155-493c-abee-41604f286e04 [following]
--2023-02-07 20:51:16--  https://doc-0s-50-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0ca9nl6mqdklkahls8l7kq3ueperu7gi/1675803075000/03249710210786382974/*/1wlHbFChZFcBgEU8knAeaUIqE0W4EaJEC?e=download&uuid=11aecb84-0155-493c-abee-41604f286e04
Resolving doc-0s-50-docs.googleusercontent.com (doc-0s-50-docs.googleusercontent.com)... 108.177.127.132, 2a00:

In [ ]:
from rayuela.base.semiring import Semiring, Real, Tropical
from rayuela.base.symbol import Sym, ε, ε_1, ε_2
from rayuela.base.misc import epsilon_filter

from rayuela.fsa.fst import FST
from rayuela.fsa.state import State, PairState

import dill
import numpy as np
from math import exp, log

from itertools import product

from frozendict import frozendict

### Subquestion a) - Log Semiring

In [ ]:
class Log(Semiring):
    def __init__(self, score):
        self.score = score

    def star(self):
        return Log(np.log(1/(1-np.exp(self.score))))

    def __float__(self):
        return float(self.score)

    def __add__(self, other):
        # TODO: implement log addition
        # For numerical stability you should use the using the log-sum-exp trick
        # https://en.wikipedia.org/wiki/LogSumExp

        return Log(np.log(np.exp(self.score) + np.exp(other.score)))

    def __mul__(self, other):
        # TODO: implement log multiplication
        return Log(self.score + other.score)

    def __repr__(self):
        return f"{round(self.score, 15)}"

    def __eq__(self, other):
        return np.allclose(float(self.score), float(other.score), atol=1e-3)

    def __hash__(self):
        return hash(self.score)

# TODO: implement the log semiring one and zero
Log.zero = Log(- np.inf)
Log.one = Log(0.0)


Log.idempotent = False
Log.cancellative = True

### Before moving to the next subquestion, we give some examples of WFSTs to illustrate how to use the library `rayuela`.

In [ ]:
# A simple language model encoded by a WFST

sem = Real # the Real semiring
fst = FST(sem) # a WFST in the Real semiring

# add transitions
fst.add_arc(State(0), Sym('formal'), Sym('formal'), State(1), Real(0.2))
fst.add_arc(State(0), Sym('natural'), Sym('natural'), State(1), Real(0.3))
fst.add_arc(State(0), Sym('learning'), Sym('learning'), State(2), Real(0.2))
fst.add_arc(State(0), Sym('data'), Sym('data'), State(3), Real(0.3))

fst.add_arc(State(1), Sym('language'), Sym('language'), State(2), Real(0.6))
fst.add_arc(State(1), Sym('languages'), Sym('languages'), State(4), Real(0.4))

fst.add_arc(State(2), Sym('is'), Sym('is'), State(5), Real(1.0))

fst.add_arc(State(5), Sym('fun'), Sym('fun'), State(6), Real(1.0))

fst.add_arc(State(3), Sym('is'), Sym('is'), State(5), Real(0.5))
fst.add_arc(State(3), Sym('are'), Sym('are'), State(5), Real(0.5))

fst.add_arc(State(4), Sym('are'), Sym('are'), State(5), Real(1.0))

# set initial weights
# if not set, it defaults to semiring zero
fst.set_I(State(0), Real(1.0))
# set final weights
fst.set_F(State(6), Real(1.0))

# visualize
fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# A simple WFSA for transliteration

sem = Real # semiring
fst = FST(sem) # initialize WFST in the chosen semiring
one, zero = sem.one, sem.zero # semiring one and zero

# add transitions
fst.add_arc(State(0), Sym('d'), Sym('data'), State(1), Real(0.5))
fst.add_arc(State(0), Sym('d'), Sym('dew'), State(5), Real(0.5))

fst.add_arc(State(1), Sym('ey'), Sym('ÃŽÂµ'), State(2), Real(0.5))
fst.add_arc(State(1), Sym('ae'), Sym('ÃŽÂµ'), State(2), Real(0.5))

fst.add_arc(State(2), Sym('t'), Sym('ÃŽÂµ'), State(3), Real(0.7))
fst.add_arc(State(2), Sym('dx'), Sym('ÃŽÂµ'), State(3), Real(0.3))

fst.add_arc(State(3), Sym('ax'), Sym('ÃŽÂµ'), State(4), one)

fst.add_arc(State(5), Sym('uw'), Sym('ÃŽÂµ'), State(6), one)

# initial and final weights
fst.set_I(State(0), one)
fst.set_F(State(4), one)
fst.set_F(State(6), one)

fst

WFSA(7 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion b) - String Transducer

In [ ]:

def string_fst(s, semiring=Log):
    """ Takes a string as input and a semiring
    and returns a transducer encoding of the
    string
    """
    fst = FST(semiring)
    one = semiring.one

    for i, char in enumerate(s):
        fst.add_arc(State(i), Sym(char), Sym(char), State(i+1), one)

    # initial and final weights
    fst.set_I(State(0), one)
    fst.set_F(State(len(s)), one)

    return fst

In [ ]:
# you can visualise the transducer
fst = string_fst("abc", semiring=Real)
fst

WFSA(4 states, <class 'rayuela.base.semiring.Real'>)

### Subquestion c) - Edit Distance Transducer


As we will not learn the weights on the arcs, you can set them to some pre-defined constants. You should choose the weights such that the pathsum that needs to be computed in e) does not diverge (this is caused by the weights of the self-loops in the transducer).



In [ ]:
def edit_distance_fst(source_alphabet, target_alphabet, semiring=Log):
    """ Takes as input two sets of characters (source and target
    alphabets) and a semiring and returns an edit-distance transducer
    """

    fst = FST(semiring)
    one = semiring.one
    eps = '\u03B5'

    fst.set_I(State(0), one)

    for i, char in enumerate(target_alphabet):
        fst.set_F(i+1, one)

    for init_state in range(len(target_alphabet)+1):
        for ci in source_alphabet.union({eps}):
            for i, co in enumerate(target_alphabet.union({eps})):

                if co != eps:
                    final_state = i
                    fst.add_arc(State(init_state), Sym(ci), Sym(co),
                                State(final_state), semiring(1))
                elif ci != eps:
                    final_state = init_state
                    fst.add_arc(State(init_state), Sym(ci), Sym(co),
                                State(final_state), semiring(1))
                else:
                    pass


    return fst

In [ ]:
fst1 = string_fst("abc", semiring=Real)
fst2 = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Real)

In [ ]:
fst = edit_distance_fst({'a', 'b', 'c'}, {'d', 'e'}, semiring=Log)

fst

WFSA(3 states, <class '__main__.Log'>)

### Subquestion d) - Composition

We provide some composition test cases for you to inspect.

In [ ]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)

In [ ]:
# FST 1
examples[0]['fst1']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# FST 2
examples[0]['fst2']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# composition
examples[0]['composition']['fst']

WFSA(2 states, <class 'rayuela.base.semiring.Real'>)

In [ ]:
# semiring
examples[0]['semiring']

rayuela.base.semiring.Real

In [ ]:
def compose(fst1, fst2):
    """ Takes as input two transducers and returns
    their composition
    """
    # The machines must be in the same semiring
    assert fst1.R == fst2.R

    comp_fst = FST(R=fst1.R)
    semiring = fst1.R
    initial_states = [(i1[0], i2[0], i3) for i1, i2, i3 in zip(fst1.I, fst2.I, {State('0')})]

    final_weights = {}
    final_states = [(f1[0], f2[0]) for f1, f2 in zip(fst1.F, fst2.F)]
    for state in final_states:
        final_weights[state] = fst1.ρ[state[0]] * fst2.ρ[state[1]]
        comp_fst.set_F(state, final_weights[state])

    # Q = initial_states
    I = initial_states
    S = initial_states # initialize a queue
    Q = [(i1[0], i2[0]) for i1, i2 in zip(fst1.I, fst2.I)]

    # computing the initial weights
    initial_weights = {}
    for state in I:
        state_pair = (state[0], state[1])
        initial_weights[state] = fst1.λ[state[0]] * fst2.λ[state[1]]

        comp_fst.set_I(state_pair, initial_weights[state])

    while S:
        # dequeue
        p_state = S.pop(0)

        # Augmentation
        input_output1 = fst1.δ[p_state[0]].copy()
        input_output1[(ε, ε_1)] = {p_state[0]: semiring.one}
        input_output2 = fst2.δ[p_state[1]].copy()
        input_output2[(ε_2, ε)] = {p_state[1]: semiring.one}

        # possible (input1, output1), (input2, output2) pairs of pairs
        io_pairs = [(io1, io2) for io1 in input_output1 \
                    if len(fst1.δ[p_state[0]][io1])>0 \
                    for io2 in input_output2 if \
                    len(fst2.δ[p_state[1]][io2])>0]


        # create loop to visit each next state
        for io_pair in io_pairs:

            # Relabeling
            q3 = p_state[2]
            a1 = Sym(str(io_pair[0][1]).replace('ε','ε_2'))
            a2 = Sym(str(io_pair[1][0]).replace('ε','ε_1'))
            filter_q3 = epsilon_filter(a1, a2, q3)

            if filter_q3 != State('⊥'):
                # previous states
                p1 = p_state[0]
                p2 = p_state[1]
                # input and output character
                i = io_pair[0][0]
                o = io_pair[1][1]
                # next states
                n1 = list(fst1.δ[p_state[0]][io_pair[0]].keys())[0]
                n2 = list(fst2.δ[p_state[1]][io_pair[1]].keys())[0]
                # weights
                w1 = list(fst1.δ[p_state[0]][io_pair[0]].values())[0]
                w2 = list(fst2.δ[p_state[1]][io_pair[1]].values())[0]
                w = w1*w2

                if (n1, n2) not in Q:
                    Q.append((n1, n2))
                    # enqueue
                    S.append((n1, n2, filter_q3))

                comp_fst.add_arc((p1, p2), i, o, (n1, n2), w)


            else:
                pass

    return comp_fst

In [ ]:
with open('test_cases.pkl', 'rb') as in_file:
    examples = dill.load(in_file)
fst1 = examples[2]['fst1']['fst']
fst2 = examples[2]['fst2']['fst']

compose(fst1, fst2)

WFSA(4 states, <class 'rayuela.base.semiring.Boolean'>)

In [ ]:
examples[2]['composition']['fst']

WFSA(5 states, <class 'rayuela.base.semiring.Boolean'>)

### Subquestion e) - Lehmann's Algorithm


In [ ]:
from IPython.core.display import display_svg
from functools import WRAPPER_ASSIGNMENTS
class Pathsum:
    def __init__(self, fsa):

        # basic FSA stuff
        self.fsa = fsa
        self.R = fsa.R
        self.N = self.fsa.num_states

        # state dictionary
        self.I = {}
        for n, q in enumerate(self.fsa.Q):
            self.I[q] = n

        # lift into the semiring
        self.W, self.W_dict, self.alpha, self.beta = self.lift()

    def lift(self):
        """ creates the weight matrix, initial weight vector,
        final weight vector from the automaton """
        W = self.R.zeros((self.N, self.N))
        W_dict = {a: {} for a in self.fsa.Sigma}
        alpha = self.R.zeros((self.N,))
        beta = self.R.zeros((self.N,))
        for a in self.fsa.Sigma:
            for b in self.fsa.Delta:
                W_dict[a][b] = self.R.zeros((self.N, self.N))
        for p in self.fsa.Q:
            alpha[self.I[p]] += self.fsa.λ[p]
            beta[self.I[p]] += self.fsa.ρ[p]
            for (a, b), q, w in self.fsa.arcs(p):
                W[self.I[p], self.I[q]] += w
                W_dict[a][b][self.I[p], self.I[q]] = w
        return W, W_dict, alpha, beta

    def lehmann(self):
        """
        Lehmann's (1977) algorithm.
        """
        R = self.W

        for j in range(len(R)):
            for i in range(len(R)):
                for k in range(len(R)):
                    R[i][k] = R[i][k] + (R[i][j]*R[j][j].star()*R[j][k])

        return R + self.R.diag(self.N)

    def _iterate(self, K):
        P = self.R.diag((self.N))
        for n in range(K):
            P += self.W @ P
        return P

    def _fixpoint(self, K=200):
        if self.fsa.R.idempotent:
            return self._iterate(self.fsa.num_states)

        diag = self.R.zeros((self.fsa.num_states, self.fsa.num_states))
        for n in range(self.fsa.num_states):
            diag[n, n] = self.fsa.R.one
        P_old = diag

        for _ in range(K):
            P_new = diag + self.W @ P_old
            P_old = P_new

        return P_old

    def fixpoint(self):

        P = self._fixpoint()
        W = {}

        for p in self.fsa.Q:
            for q in self.fsa.Q:
                if p in self.I and q in self.I:
                    W[p, q] = P[self.I[p], self.I[q]]
                elif p == q:
                    W[p, q] = self.R.one
                else:
                    W[p, q] = self.R.zero

        return frozendict(W)

In [ ]:
pathsum=Pathsum(examples[3]['fst1']['fst'])
print(pathsum.fixpoint())
print(pathsum.lehmann())

frozendict.frozendict({(0, 0): 1.204687055617909, (0, 1): 0.049703748026656, (0, 2): 0.238528037012346, (1, 0): 0.423848011191222, (1, 1): 1.064607810824038, (1, 2): 0.083921906215862, (2, 0): 0.297575363071423, (2, 1): 0.220654517754702, (2, 2): 1.058919921888142})
[[1.210030479232752 0.052036796373853 0.249169852930657]
 [0.442313809320239 1.071795077141898 0.118150643517558]
 [0.315208576189901 0.226634327641494 1.085202126621377]]


### Subquestion f) - Normalizer


We provide some test cases for you to inspect.

In [ ]:
examples[0]['fst1']['Z'], examples[0]['fst2']['Z'], examples[0]['composition']['Z']

(0.00138425, 0.000997195294118, 7.07192371e-07)

In [ ]:
def normalizer(fst):
    """ Takes as input a finite-state transducer and
    returns its normalizer
    """

    pathsum = Pathsum(fst)
    # TODO: implement normalizer
    lehm = pathsum.lehmann()
    Z = pathsum.alpha @ lehm @ pathsum.beta

    return Z

In [ ]:
normalizer(examples[1]['fst1']['fst']), normalizer(examples[1]['fst2']['fst']), normalizer(examples[1]['composition']['fst'])

(Tropical(66), Tropical(77), Tropical(166.0))

In [ ]:
examples[1]['fst1']['Z'], examples[1]['fst2']['Z'], examples[1]['composition']['Z']

(Tropical(66), Tropical(77), Tropical(166.0))

### Subquestion g) - Log-likelihood

weight of
use the function that was coded previously

use composition and the normalizer. We need to figure out what we have to compose.

In [ ]:
def log_likelihood(source_alphabet, target_alphabet, source_str, target_str):
    """ Takes as input the source and target alphabets (sets of characters)
    and a pair of source and target strings and returns the log-likelihood
    """

    # TODO: implement log-likelihood

    raise NotImplementedError

### Subquestion h) - Decoding

input string as a transducer, then compose with the edit distance transducer.
after you get the composition figure out which is the weight of the path with the highest score.

You can use lehmann.

In [ ]:
def to_semiring(fst, semiring):
    """ Lifts an WFST into a different semiring
    """
    oone, one = fst.R.one, semiring.one
    nfst = FST(semiring)

    # initial weights
    for q, w in fst.I:
        if w == oone:
            nfst.set_I(q, one)
        else:
            nfst.set_I(q, semiring(w.score))

    # final weights
    for q, w in fst.F:
        if w == oone:
            nfst.set_F(q, one)
        else:
            nfst.set_F(q, semiring(w.score))

    # arcs
    for p in fst.Q:
        for (a, b), q, w in fst.arcs(p):
            if w == oone:
                nfst.add_arc(p, a, b, q, one)
            else:
                nfst.add_arc(p, a, b, q, semiring(w.score))

    return nfst

In [ ]:
def decode(in_str, edit_distance_fst):
    """ Gets as input a source string and an
    edit-distance transducer and returns the weight of the most
    probable target string
    """
    fst = compose(string_fst(in_str), edit_distance_fst)
    # I should  use the arctic semiring to compute this task but it's not defined in the rayuela library
    artic_fst = to_semiring(fst, Arctic)
    max_weight = normalizer(artic_fst)

    return max_weight